In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('dataset/train')
label_types = os.listdir('dataset/train')

print(label_types)


['bicepscurl', 'squat']


Preparing Training Data

In [8]:
rooms = []

for item in dataset_path:
    # Getting all the file names
    all_rooms = os.listdir('dataset/train' + '/' + item)

    #Adding them to the list
    for room in all_rooms:
        rooms.append((item , str('dataset/train' + '/' + item) + '/' + room))
    
#Building a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])


In [19]:
train_df.shape

(79, 2)

In [24]:
df = train_df.loc[:,['video_name', 'tag']]
df
df.to_csv('train.csv')

Preparing test data

In [14]:
dataset_path = os.listdir('dataset/test')

rooms = []

for item in dataset_path:
    #Getting all the file names
    all_rooms = os.listdir('dataset/test' + '/' + item)

    #Adding them to the list
    for room in all_rooms:
        rooms.append((item, str('dataset/test' + '/' + item) + '/' + room))

test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])



In [18]:
test_df.head()
test_df.tail()
test_df.shape

(12, 2)

In [25]:
df = test_df.loc[:,['video_name', 'tag']]
df
df.to_csv('test.csv')

Installing tensorflow

In [ ]:
#%pip install git+https://github.com/tensorflow/docs

In [28]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os


Data preparation

In [29]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"Total videos for trainging : {len(train_df)}")
print(f"Total videos for testing : {len(test_df)}")

Total videos for trainging : 79
Total videos for testing : 12


In [30]:
train_df.sample(10)

,Unnamed: 0,video_name,tag
16,16,dataset/train/bicepscurl/barbell biceps curl_2...,bicepscurl
18,18,dataset/train/bicepscurl/barbell biceps curl_2...,bicepscurl
6,6,dataset/train/bicepscurl/barbell biceps curl_1...,bicepscurl
0,0,dataset/train/bicepscurl/barbell biceps curl_1...,bicepscurl
5,5,dataset/train/bicepscurl/barbell biceps curl_1...,bicepscurl
67,67,dataset/train/squat/squat_2.MOV,squat
58,58,dataset/train/squat/squat_11.mp4,squat
62,62,dataset/train/squat/squat_15.mp4,squat
24,24,dataset/train/bicepscurl/barbell biceps curl_3...,bicepscurl
35,35,dataset/train/bicepscurl/barbell biceps curl_4...,bicepscurl


Feed the videos to a network

In [31]:
IMG_SIZE = 224

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

def load_video(path, max_fames = 0, resize = (IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]

            if len(frames) == max_fames:
                break
    
    finally:
        cap.release()
    
    return np.array(frames)


Feature Extraction

In [34]:
def build_feature_extractor() -> None: 
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

In [35]:
feature_extractor = build_feature_extractor()

ValueError: object __array__ method not producing an array